# Christmas Elves

### Rules
1. Roll the die to wrap a present
2. Must roll equal to or *higher than* the wrapping difficulty

### Ideas to Enhance Difficulty/Complexity
- Have to roll the exact number
- Reduce number of turns
- Add one-off events
- Give each player's elf a special power
- Add more attributes/complexity to Presents

In [ ]:
import os
import numpy as np
import pandas as pd
import json
from prodict import Prodict
import random

with open('presents.json', 'r') as file:
    presents_json = Prodict.from_dict(json.load(file))



class Present():

    def __init__(self, name, difficulty):
        self.name = name
        self.difficulty = difficulty

diff_levels = {
    "Easy": [.25, 12, "Easy"],
    "Medium": [.4, 12, "Medium"],
    "Hard": [.45, 12, "Hard"]
}

wrapping_level_d12 = {
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 4,
    "6": 5,
    "7": 5,
    "8": 4,
    "9": 4,
    "10": 3,
    "11": 2,
    "12": 1
}

wrapping_level_d20 = {
    
}

test_present_list=[]
for idx, present in enumerate(random.sample(presents_json.presents, k=12)):
    test_present_list.extend(
        [Present(name=present, difficulty=idx+1)]*wrapping_level_d12[str(idx+1)]
    )

class Game():

    def __init__(self, n_players, present_list, diff_lvl, d):
        self.n_players = n_players
        self.turns = diff_lvl[1]*self.n_players
        self.d = d
        self.diff_lvl = diff_lvl
        self.n_presents = self.generate_presents(present_list=test_present_list)
        #self.n_presents = self.generate_presents(present_list=[Present(name=present, difficulty=random.randint(self.d//4, self.d-1)) for present in present_list.presents], diff_lvl=diff_lvl)
        self.presents_wrapped = 0
        self.active_key = '0'
        self.game_results = pd.DataFrame(None, columns=['Difficulty_Level', 'N_Players', 'N_Turns', 'Die', 'Present_List', 
             'N_Presents', 'Presents_Wrapped', 'Current_Present_Name', 'Current_Present_Difficulty',
            'Player_Roll', 'Wrapped_YN', 'Player_Won_YN', 'Game_End_YN']
        )

    def turn_end(self):
        self.turns -= 1
        #print(f'Turns Left: {self.turns}')  

    def generate_presents(self, present_list):
        present_dict = dict()
        for idx, present in enumerate(random.choices(present_list, k=int(self.turns*self.diff_lvl[0]))):
            present_dict[str(idx)] = Prodict.from_dict({"Name": present.name, "Difficulty": present.difficulty})
        
        return Prodict.from_dict(present_dict)

    def take_turn(self, current_present):
        #print(f'\n\nTurn {self.turns}') #DEBUG
        #print(f'Active Key: {self.active_key}') #DEBUG
        player_roll = random.randint(1, self.d)
        if player_roll >= current_present['Difficulty']:
            #Iterate to next present
            #print(f'Present successfully wrapped! (Present difficulty: {current_present["Difficulty"]}. Player rolled {player_roll}!)') #DEBUG
            self.presents_wrapped += 1
            self.active_key = str(int(self.active_key)+1) #Advance one key
            self.check_win_conditions(current_present, player_roll, wrapped=True)
            self.turn_end()
        else:
            #print(f'Roll failed! (Present difficulty: {current_present["Difficulty"]}. Player rolled {player_roll}!)') #DEBUG
            self.check_win_conditions(current_present, player_roll, wrapped=False)
            self.turn_end()

    def start_sim(self):
        self.game_over = False
        
        while self.game_over == False:
            self.take_turn(self.n_presents[self.active_key])
            
        return self.game_results
        #print('\n\nSimulation Complete!') #DEBUG

    def check_win_conditions(self, current_present, player_roll, wrapped):
        player_won = False
        if self.presents_wrapped == len(self.n_presents):
            #print(f'Player Wins! {self.presents_wrapped} presents wrapped!') #DEBUG
            player_won = True
            self.game_over = True
        elif self.turns < (len(self.n_presents)-self.presents_wrapped):
            #print(f'Player Loses! Insufficient turns left to wrap remaining presents!') #DEBUG
            player_won = False
            self.game_over = True
        else:
            player_won = False
            self.game_over = False            
            pass
        round_results = pd.DataFrame([self.diff_lvl[2], self.n_players, self.turns, self.d, str(self.n_presents), len(self.n_presents), self.presents_wrapped, current_present["Name"],current_present["Difficulty"], player_roll, wrapped, player_won, self.game_over])
        rr_t = round_results.transpose()
        rr_t.columns = self.game_results.columns
        self.game_results = pd.concat([self.game_results, rr_t], axis=0)

def run_multiple_sims(n_iterations=1000, n_players=4, present_list=None, diff_lvl=None, d=6):
    df_final_results = pd.DataFrame(
        None, 
        columns=['Difficulty_Level', 'N_Players', 'N_Turns', 'Die', 'Present_List', 
                 'N_Presents', 'Presents_Wrapped', 'Current_Present_Name', 'Current_Present_Difficulty',
                'Player_Roll', 'Wrapped_YN', 'Player_Won_YN', 'Game_End_YN']
    )
    for i in range(n_iterations):
        new_game = Game(
        n_players=n_players, 
        present_list=present_list, 
        diff_lvl=diff_lvl,
        d=d
        )
        df_final_results = pd.concat([df_final_results, new_game.start_sim()], axis=0)
        df_final_results.reset_index(inplace=True, drop=True)

    if len(df_final_results.query('Game_End_YN == True')) != n_iterations:
        raise Exception(f'Iteration Error: Expected number of iterations was {n_iterations}. Number of iterations this cycle was {len(df_final_results.query("Game_End_YN == True"))}')
    else:
        game_end = df_final_results.query('Game_End_YN == True')
        player_win_pct = round(len(game_end.query("Player_Won_YN == True"))/len(game_end),3)
        print(f'Player Win: {round(player_win_pct*100,1)}%')
        return df_final_results, pd.Series([n_iterations, n_players, player_win_pct, diff_lvl[2]])


In [ ]:
all_metadata = []
all_data = pd.DataFrame(None)
for i in range(10):
    data, metadata = run_multiple_sims(n_iterations=1000, n_players=3, present_list=presents_json, diff_lvl=diff_levels['Easy'], d=12)
    all_data = pd.concat([all_data, data], axis=0)
    all_metadata.append(metadata)

df_metadata = pd.DataFrame(all_metadata)
df_metadata.columns=['n_iterations', 'n_players', 'win_pct', 'diff_lvl']
print(f'Average Win Pct: {df_metadata.win_pct.mean()}') #DEBUG

with open('metadata1.csv', 'a') as f:
    df_metadata.to_csv(f, index=False, sep='\t', header=False)
with open('output1.csv', 'a') as df:
    all_data.to_csv(df, sep='\t', index=False, header=False)

# Analysis Results
### Easy Difficulty, D12

**One Player**
- 

**Two Players**
-

**Three Players**


**Four Players**
- 93.84% wins
- 10,000 iterations

In [ ]:
test_metadata = pd.read_csv('metadata3.csv', sep='\t')
test_metadata.columns = ['n_iterations', 'n_players', 'win_pct', 'diff_lvl']
all_means = test_metadata.loc[:, ['diff_lvl', 'n_players','win_pct']].groupby(['diff_lvl', 'n_players']).mean()
all_means

# Analysis Portion

In [3]:
import os
import numpy as np
import pandas as pd
import json

pd.options.display.max_columns = None
pd.options.display.max_rows = None

def extract_difficulty(present_list):
    big_diff_list = []
    for item in present_list:
        item_dict = eval(item)
        diff_list = []
        for idx, key in enumerate(item_dict):
            diff_list.append(item_dict[key]['Difficulty'])
        diff_list.sort()
        dl = str(tuple(diff_list))
        summ = np.array(diff_list).sum()
        big_diff_list.append([dl, summ])

    return pd.DataFrame(big_diff_list, columns=['Difficulties', 'Sum'])

df_raw = pd.read_csv('output1.csv', sep='\t')
df_raw.columns = ['Difficulty_Level', 'N_Players', 'N_Turns', 'Die', 'Present_List', 
                 'N_Presents', 'Presents_Wrapped', 'Current_Present_Name', 'Current_Present_Difficulty',
                'Player_Roll', 'Wrapped_YN', 'Player_Won_YN', 'Game_End_YN'] #This is only for output1.csv
print(f'All Rows: {df_raw.shape}')
df_filtered = df_raw.query('Game_End_YN == True')
print(f'Game End Only: {df_filtered.shape}')
data = df_filtered.query('Player_Won_YN == True')
print(f'Player Won Only: {data.shape}')

#Find top difficulty combinations
df_presents = extract_difficulty(data.Present_List)
df_presents['Count'] = [1 for i in range(0, len(df_presents))]
df_grp = df_presents.groupby(['Difficulties', 'Sum']).count()
df_grp.query('Count >= 5').sort_values(by=['Count'], ascending=False)

All Rows: (212203, 13)
Game End Only: (10000, 13)
Player Won Only: (9235, 13)


,,Count
Difficulties,Sum,
"(3, 4, 5, 6, 7, 8, 8, 9, 10)",60,10
"(2, 3, 4, 5, 6, 7, 7, 8, 10)",52,5
"(2, 3, 5, 5, 6, 6, 7, 9, 9)",52,5
"(2, 4, 5, 6, 6, 7, 8, 10, 11)",59,5
"(3, 4, 5, 6, 6, 6, 6, 7, 9)",52,5
"(3, 4, 5, 6, 6, 7, 7, 9, 9)",56,5
"(3, 4, 5, 6, 7, 7, 8, 9, 10)",59,5
"(4, 5, 5, 6, 7, 7, 8, 9, 10)",61,5
"(4, 5, 6, 7, 7, 7, 8, 10, 11)",65,5


In [ ]:
output4 = pd.read_csv('output4.csv', sep='\t')
output4.columns.tolist()

In [4]:
turns_pres = df_raw.query('Game_End_YN == True and Player_Won_YN == True and Difficulty_Level == "Easy"').loc[:, ['N_Players', 'N_Turns', 'N_Presents']]
turns_pres['Turn_to_Presents'] = turns_pres.N_Turns/turns_pres.N_Presents
turns_pres.head()

,N_Players,N_Turns,N_Presents
17,3,18,9
36,3,18,9
49,3,24,9
83,3,3,9
96,3,24,9


In [7]:
turns_pres.Turn_to_Presents.describe()

count    9235.000000
mean        1.874511
std         0.708336
min         0.000000
25%         1.444444
50%         2.000000
75%         2.444444
max         3.111111
Name: Turn_to_Presents, dtype: float64

#### 